# Amino Acid sequence generation

The `data.csv` file contanins a database of known positive protein-protein interactions. The problem lies in the fact that these proteins are registered as gene symbols, whereas any feature extraction operation requires the complete amino acid sequence of the protein.

The goal of this notebook is to extract all the gene symbols present in the HPRD database (`data/symbol_data.csv`) and encode them as AA sequences using the NCBI repository.

This notebook should be run before `generateCompleteDB.ipynb`

## Imports:

In [3]:
import pandas as pd
import re
import json
from Bio import SeqIO, Entrez

## Querying the NCBI repository:

In [5]:
# setting up the email address for Entrez
Entrez.email = 'omaratyqy@gmail.com'

# read the database into a pandas dataframe
df = pd.read_csv('data/symbol_data.csv')

# remove rows with self-interactions
N_old = len(df)
df = df[df['Interactor 1 RefSeq id'] != df['Interactor 2 RefSeq id']]
N_new = len(df)

# display number of removed rows as a percentage
print(f"Removed {N_old - N_new} rows - ({round((N_old - N_new) / N_old * 100, 2)}%)")

# create a list of RefSeq IDs
refseq_ids = list(set(df['Interactor 1 RefSeq id'].tolist() + df['Interactor 2 RefSeq id'].tolist()))

# create an empty dictionary to store the sequences
sequences = {}

# retrieve the protein sequences in batches of 500
for i in range(0, len(refseq_ids), 500):
    refseq_ids_batch = refseq_ids[i:i+500]
    
    # convert the list of RefSeq IDs to a search term for Entrez
    search_term = ' OR '.join(str(refseq_id) for refseq_id in refseq_ids_batch)

    # search for the RefSeq IDs in the NCBI protein database
    handle = Entrez.esearch(db='protein', term=search_term, retmax=500)
    record = Entrez.read(handle)

    # retrieve the protein sequences for the matching RefSeq IDs
    if record['IdList']:
        protein_ids = record['IdList']
        handle = Entrez.efetch(db='protein', id=protein_ids, rettype='fasta', retmode='text')
        seq_records = list(SeqIO.parse(handle, 'fasta'))

        # store the sequences in the dictionary
        for seq_record in seq_records:
            gene_symbol = seq_record.id.split('|')[0]
            sequences[gene_symbol] = str(seq_record.seq)

    # print the current length of the dictionary divided by the total number of RefSeq IDs as a progress indicator
    progress = round(len(sequences) / len(refseq_ids) * 100, 2)
    print(f"Current progress: {progress}%")

# print the result
print(f"Total number of sequences: {len(sequences)}")

# write the sequences to a json file for later use
import json
with open('data/sequences.json', 'w') as f:
    json.dump(sequences, f)

Removed 2160 rows - (5.5%)


URLError: <urlopen error [Errno 11002] getaddrinfo failed>

##  Generating the AA database:

In [ ]:
# read the sequences from the json file
with open('data/sequences.json', 'r') as f:
    sequences = json.load(f)

# read the database into a pandas dataframe
df = pd.read_csv('data/symbol_data.csv')

# number of skipped sequences
skipped = 0

# create a new dataframe to store the sequences
df_sequences = pd.DataFrame(columns=['Sequence 1', 'Sequence 2'])

# iterate over the rows of the database
for index, row in df.iterrows():
    # get the id for the two interactors
    try:
        seq1 = sequences[row['Interactor 1 RefSeq id']]
        seq2 = sequences[row['Interactor 2 RefSeq id']]
    except KeyError:
        skipped += 1
        continue

    # add the sequences to the dataframe
    df_sequences = pd.concat([df_sequences, pd.DataFrame([[seq1, seq2]], columns=['Sequence 1', 'Sequence 2'])])

    # print the progress as a percentage
    progress = round(index / len(df) * 100, 2)
    print(f"Current progress: {progress}%")


In [ ]:
# number of sequences in the original database
N_original = len(df)
print(f"Number of rows in the original database: {N_original}")

# number of sequences in the new database
N_aa_sequence = len(df_sequences)
print(f"Number of rows in the new database: {N_aa_sequence}")

# print the number of skipped sequences
print(f"Skipped sequences: {skipped} | {round(skipped / N_original * 100, 2)}%")

# Saving the new database:

In [ ]:
# save the dataframe to a csv file
df_sequences.to_csv('data/sequence_data.csv', index=False)